# Zapping in & Zapping Out
> "Entering & Exiting the yvBoost Yearn Vault"

- toc:true
- branch: master
- badges: true
- comments: false
- author: Scott Simpson
- categories: [Curve, Yearn]
- hide: true  


In [1]:
#hide
#Imports & settings
!pip install plotly --upgrade
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline
#%load_ext google.colab.data_table
%load_ext rpy2.ipython
%R options(tidyverse.quiet = TRUE)
%R options(lubridate.quiet = TRUE)
%R options(jsonlite.quiet = TRUE)
%R suppressMessages(library(tidyverse))
%R suppressMessages(library(lubridate))
%R suppressMessages(library(jsonlite))
%R suppressMessages(options(dplyr.summarise.inform = FALSE))


     |████████████████████████████████| 23.9 MB 13 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


dplyr.summarise.inform,[RTYPES.NILSXP]


In [9]:
#hide
%%R

#Grab base query from Flipside
df = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/2d839745-a264-4ada-9e02-48e8f603282b/data/latest', simplifyDataFrame = TRUE)

#fix the column names
names(df)<-tolower(names(df))

#Change the date to date format
df$block_timestamp <- parse_datetime(df$block_timestamp)

#Grab the token labels
labels <- read_csv("https://github.com/scottincrypto/analytics/raw/master/data/yvBoost_tokenlist.csv", show_col_types = FALSE)

#Join the new symbols into the main df
df <- df %>% select(-symbol) %>%
  left_join(labels) %>%
  mutate(symbol = replace_na(symbol, "ETH/WETH"),
         group = replace_na(group, "ETH/WETH"),
         simple_group = replace_na(simple_group, "ETH/WETH")) 

#Add usd pricing for TUSD and BUSD
df <- df %>%
  mutate(amount_usd = if_else(contract_address %in% c('0x0000000000085d4780b73119b644ae5ecd22b376',
                                                      '0x4fabb145d64652a948d72533023f6e7a623c7c53')
                              , amount, amount_usd))

#Add a date field
df <- df %>%
  mutate(week = floor_date(block_timestamp, unit = 'week'),
         date = floor_date(block_timestamp, unit = 'day'))

#grab the yveCRV prices
yveCRV_prices = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/f13f3e31-f6e2-4862-9f24-254095098fb5/data/latest', simplifyDataFrame = TRUE)
names(yveCRV_prices)<-tolower(names(yveCRV_prices))
yveCRV_prices$date <- parse_datetime(yveCRV_prices$date)

#join & update
df <- df %>% 
  left_join(yveCRV_prices %>% select(date, price)) %>%
  mutate(amount_usd = if_else(symbol == 'yveCRV-DAO', amount * price, amount_usd)) #%>%
 # select(-price)

#Sankey setup
# LHS is Zapin
# Middle is yvBoost Vault
# RHS is Zapout
# 
# Link table is
# Symbol-Amount_USD aggregation (source node 0 added manually)
# Symbol-Destination_Label-Amount_USD

#Nodes are:
# Bridge In
# 4 x Currencies (wallets)
# 10 x Groups (output groupings)

#replace nas with Unknown
#df$Label <- replace_na(df$Label, "Unknown")
#df$Class <- replace_na(df$Class, "Unknown")

#create the RHS of the sankey link table
rhs_link <- df %>%
  filter(zap_function == 'Zapout') %>%
#  filter(zap_function %in% c('Zapout', 'Withdraw')) %>%
  mutate(source = "yvBoost Vault") %>%
  group_by(source, group) %>%
  summarise(total = sum(amount_usd, na.rm = TRUE)) %>%
  rename(target = group) %>%
  mutate(target = paste(target, 'out', sep = ' ')) %>%
  ungroup()


#create the LHS of the sankey link table
lhs_link <- df %>%
  filter(zap_function == 'Zapin') %>%
#  filter(zap_function %in% c('Zapin', 'Deposit')) %>%
  mutate(target = "yvBoost Vault") %>%
  group_by(group, target) %>%
  summarise(total = sum(amount_usd, na.rm = TRUE)) %>%
  rename(source = group) %>%
  mutate(source = paste(source, 'in', sep = ' ')) %>%
  ungroup()

#Join the table together
link_table <- lhs_link %>% bind_rows(rhs_link) 

#Create a list of the nodes
nodes <- link_table %>% rename(node = source) %>%
  distinct(node) %>% 
  bind_rows(link_table %>% rename(node = target) %>% distinct(node) %>% filter(node != 'yvBoost Vault')) %>% 
#  distinct(node) %>%
  mutate(index = row_number()-1)

#join the index back into the link table
link_table <- link_table %>%
  left_join(nodes, by=c("source" = "node")) %>%
  rename(source_index = index)

link_table <- link_table %>%
  left_join(nodes, by=c("target" = "node")) %>%
  rename(target_index = index)

Joining, by = "contract_address"
Joining, by = "date"


# Yearn yVaults, yveCRV-DAO and yvBOOST

## Yearn yVaults
Yearn is a DeFi protocol which automates yield farming.  Users have tokens which they want to hold - Yearn puts those tokens to work by finding the best yield farming opportunities across DeFi.  It does this in a gas efficient way for the user, so even small deposits can get decent returns over time.

Yearn users deposit their tokens into yVaults, and receive a token in return which is proportional to their share of the vault capital.  A yVault is a smart contract with one ore more Strategies sitting behind it.  The Strategies are the yield-farming recipes which are created by clever humans (Strategists) and monitored & managed by bots (Keepers).  The yVault contains logic which automatically allocates the vault deposits to whichever combination of Strategies gives the best return for the users.  The rewards from the yield farming accrue into the vault, so the value of the vault token is always increasing.  When a user withdraws from the yVault, they get more tokens than they deposited.  This additional amount is the yield the vault has earned on their behalf.

Each yVault is structured around a particular underlying token - there are vaults for Eth, USDC, WBTC and many others.  Users can deposit in the yVault native token, or they can deposit using any other token & take advantage of Zaps.  Zaps are smart contracts which take an input token and swap it for the underlying token in a gas efficient manner.  The swap may occur via a number of dexs or dex aggregators, but this is abstracted away for the user.  There is a similar feature when withdrawing - the user can withdraw the underlying token from a yVault, or choose to receive their funds in ETH, WBTC, DAI, USDC or USDT.  It's important to know that whatever token the user deposits  or withdraws, they maintain price exposure to the *underlying token* of the vault whilst deposited.  Fun fact - Zaps aren't just used by Yearn - they are the tech which underpins the swap & deposit functions on [Zapper](https://zapper.fi), and are used by other protocols like [Pickle Finance](https://pickle.finance)

## veCRV-DAO yVault
Now the veCRV-DAO yVault is a little different to the others.  It starts with the CRV token, the governance & reward token from [Curve](https://curve.fi).  Curve is a dex which specialises in stableswaps - swaps between tokens which have approximately the same value.  Examples are ETH/stETH or swaps between dollar pegged stablecoins.  Curve has optimised their swap code to make these swaps efficient from both a liquidity impact and fee perspected - see this [post](https://scottincrypto.github.io/analytics/curve/2021/09/19/Curve_Stableswaps.html) for a further exploration of Curve & stablecoin swaps.

The CRV token has voting rights in the Curve DAO which makes decisions on the Curve protocol - things like fees, LP rewards, swap parameters and pools launched.  In some Curve pools, liquidity providers receive CRV tokens to incentivise liquidity in the pools.  CRV tokens are also available on the open market.  To encourage users to stay as CRV hodlers, there is a facility to lock CRV tokens into the CRV DAO for a fixed period of up to 4 years.  Users receive veCRV tokens (voting escrow Curve Tokens) for doing this, and more tokens are received the longer the locking period.  veCRV holders can still participate in governance voting, they receive 50% of Curve trading fees and they qualify for boosted rewards (up to 2.5x) when they provide liquidity in Curve.

The fees generated for veCRV holders are collected in the form of 3CRV tokens (shares in the [Curve tripool](https://curve.fi/3pool), which can be redeemed for stablecoins if desired.  Fee distribution for veCRV holders happens weekly and users need to collect these manually and pay the gas cost for the transactions.

In true Yearn fashion there is a vault & a strategy to maximise the returns from this CRV locking process.  This is the veCRV-DAO yVault and it's different to the other vaults in that you *can't withdraw your tokens*.  Yearn takes CRV tokens and locks them with the CRV DAO for the maximum 4 year period and continually renews this lock.  This maximises the veCRV returns to the yVault.  In addition, all Yearn vaults send 10% of earned CRV into this vault for additional boost.  The returns to the users are in the form of the 3CRV tokens earned by the veCRV - like the CRV staking contract, these are collectable weekly as an income stream, and must be collected manaully.  yVault depositors receive yveCRV-DAO tokens as their share in the vault.

## yvBoost

Finally we get to the yvBoost yVault.  This vault builds on the veCRV-DAO yVault, but automates the process of collecting the weekly rewards.  The Strategy behind this vault collects the 3CRV rewards each week, swaps them for more yveCRV then deposits them back into the vault.  The yvBoost vault is a standard Yearn yVault - you can withdraw part or all of your outstanding deposit and any accrued gains at any time.  Working in the native tokens of the yVault, users can deposit or withdraw yveCRV-DAO tokens.  Alternatively, users can take advantage of the Zap function and deposit pretty much any ERC-20 token into the vault.  They can also withdraw using Zap and collect WETH, WBTC, DAI, USDT or USDC.  As the Zap conversions occur on the way in and out, the user maintains price exposure to the yveCRV-DAO tokens whilst deposited in the vault.  This post will look at what assets users choose to deposit and withdraw with in the yvBoost yVault, paying particular attention to what happens when they choose to Zap tokens other than yveCRV-DAO into or out of the vault.

# Deposits, Withdrawals & Zaps for yvBoost

There are many ways in & out of Yearn yVaults.  Users can deposit or withdraw from the interface on the https://yearn.finance, using yveCRV or other tokens with Zaps.  Users can also deposit via Pickle Finance, or via any number of other 3rd party interfaces, or even by directly interacting with contracts themselves. We saw 244 different contract calls which deposited or withdrew from the yvBoost vault.  This post will only look at transactions from:
- The standard deposit and withdraw function calls in the [yvBoost yVault](https://etherscan.io/address/0x9d409a0a012cfba9b15f6d4b36ac57a46966ab9a)
- Zapin deposits, via the [yVault_ZapIn_V4](https://etherscan.io/address/0x92be6adb6a12da0ca607f9d87db2f9978cd6ec3e) and [yVault_ZapIn_V3](https://etherscan.io/address/0x42d4e90ff4068abe7bc4eab838c7de1d2f5998a3) contracts
- Zapout withdrawals, via the [yVault_ZapOut_V3_0_1](https://etherscan.io/address/0xd6b88257e91e4e4d4e990b3a858c849ef2dfde8c) and [yVault_ZapOut_V2](https://etherscan.io/address/0xc5b49abf8b164472be9486767af6b1a5586b5609) contracts

At a high level, it's useful to understand how much of the yvBoost yVault deposit & withdrawal actions are happening via the native functions (using yveCRV-DAO) or via Zaps (using different tokens).  The graph below shows this.  The overwhelming majority of transactions occur using yveCRV-DAO tokens via the native Deposit & Withdrawal functions:

In [10]:
#hide_input
df_p = %R df %>% group_by(zap_function) %>% summarise(total = sum(amount_usd, na.rm = TRUE)) %>% arrange(desc(total))
fig = px.bar(df_p
             , x = "zap_function"
             , y = "total"
             , color = 'zap_function'
             , labels=dict(zap_function="Operation", total="USD Amount")
             , title= "Deposits, Withdrawals and Zaps in & out of yvBoost"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(showlegend=False)
fig.update_yaxes(title_text='Amount (USD)')
fig.update_xaxes(title_text='Operation')
fig.show()

# Zooming in on the Zaps

The following diagram shows the Zap In & Out transactions to & from the yvBoost yVault.  We can see, as we did in the graph above, that Zap In transactions outweigh Zap Out transactions by a factor of 4.  This makes sense - the vault has a significant TVL ($11m USD at the time of this post) and so people deposit more than they withdraw.  Remember for the Zap in side, we can deposit ETH or any ERC-20 token, and the Zap will convert it to yveCRV-DAO and deposit in the vault.  We can see stablecoins are well represented (BUSD, USDC, DAI, TUSD), and ETH/WETH as well.  CRV is the other significant volatile asset.  On the exit side, where we are limited to withdrawing only 5 types of token, ETH/WETH is by far the largest.  We will have a closer look at the makeup of Zap in & out transaction in the next section.

In [11]:
#hide_input
# Sankey by Label
label = %R nodes %>% select(node)
source = %R link_table %>% select(source_index)
target = %R link_table %>% select(target_index)
value = %R link_table%>% select(total)

label = label['node'].to_list()
source = source['source_index'].to_list()
target = target['target_index'].to_list()
value = value['total'].to_list()

# data to dict, dict to sankey
link = dict(source = source, target = target, value = value)
node = dict(label = label, pad=50, thickness=15)
data = go.Sankey(link = link, node=node)
fig = go.Figure(data)
fig.update_layout(width=800, height=800/1.618, template="simple_white", title="Zap Transactions In & Out of the yvBoost Vault (amounts in USD)", hovermode = 'x')
fig.show()

## Zap In Transactions - Which Tokens?

The following graph shows all the Zap in tokens in more detail.  In the previous graph, the long tail was rolled up to "other" for easier viewing.  As we saw in the network diagram above, dollar pegged stablecoins are most popular with WETH/ETH the 2nd largest category outright.  An array of less popular stablecoins and defi tokens make up the long tail.  Notice the odd coin out?  Somehow a user has managed to deposit yveCRV-DAO via a Zap transaction.

In [12]:
#hide_input
#time plot by week for inputs
df_p = %R df %>% filter(zap_function == 'Zapin') %>% group_by(symbol) %>% summarise(total = sum(amount_usd, na.rm = TRUE)) %>% arrange(desc(total))
fig = px.bar(df_p
             , x = "symbol"
             , y = "total"
             , color = 'symbol'
             , labels=dict(week="Week", total="USD Amount", simple_group='Zap In Token')
             , title= "Tokens Deposited to yvBoost Vault via Zap"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(showlegend=False)
fig.update_yaxes(title_text='Amount (USD)')
fig.update_xaxes(title_text=None)
fig.show()

## Zap Out Transactions - Which Tokens?

The following graph shows all the Zap out tokens in more detail.  ETH/WETH withdrawals are by far the most common, with the 3 stablecoins following up.  Only a very small amount is withdrawn as WBTC using Zaps.  Again, we also see a withdrawal transaction in yveCRV-DAO via Zap - a curious outlier.



In [13]:
#hide
#time plot by week for inputs
df_p = %R df %>% filter(zap_function == 'Zapout') %>% group_by(symbol) %>% summarise(total = sum(amount_usd, na.rm = TRUE)) %>% arrange(desc(total))
fig = px.bar(df_p
             , x = "symbol"
             , y = "total"
             , color = 'symbol'
             , labels=dict(week="Week", total="USD Amount", simple_group='Zap In Token')
             , title= "Tokens Withdrawn from yvBoost Vault via Zap"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(showlegend=False)
fig.update_yaxes(title_text='Amount (USD)')
fig.update_xaxes(title_text=None)
fig.show()

# Zap In over Time

Next we see what users Zap into the yvBoost over time.  The graph below looks at the Zap In transactions, with the categories grouped up to aid with interpretation.  All of the USD pegged stablecoins are labelled as such, and the long tail of DeFi tokens are labelled as "other".  We can see a large influx into the yvBoost yVault in May 2021.  There may be a couple of reasons for this - firstly, the yvBoost yVault was launched and may have attracted some early attention, and secondly, this was the time when crypto prices crashed across the board.  This may have driven the large deposits of stablecoins in mid-May, as users looked for safe haven yields when volatile asset values were diving.  Zap In deposits droppped to near zero from mid-June through to September, when a pickup of activity occurred.

In [14]:
#@title
#hide_input
#time plot by week for inputs
df_p = %R df %>% filter(zap_function == 'Zapin') %>% group_by(week, simple_group) %>% summarise(total = sum(amount_usd, na.rm = TRUE))
fig = px.bar(df_p
             , x = "week"
             , y = "total"
             , color = 'simple_group'
             , labels=dict(week="Week", total="USD Amount", simple_group='Zap In Token')
             , title= "Tokens Deposited to yvBoost Vault via Zap"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='Amount (USD)')
fig.update_xaxes(title_text=None)
fig.show()

# Zap Out over Time

The corresponding chart for Zap Out transactions is below, with the same groupings of stablecoins shown.  Aside from one large week of withdrawals at the end of April, Zap Out withdrawals were very quiet until June 2021.  Withdrawals have been steadily trending down since that time.



In [15]:
#@title
#hide_input
#time plot by week for outputs
df_p = %R df %>% filter(zap_function == 'Zapout') %>% group_by(week, simple_group) %>% summarise(total = sum(amount_usd, na.rm = TRUE))
fig = px.bar(df_p
             , x = "week"
             , y = "total"
             , color = 'simple_group'
             , labels=dict(week="Week", total="USD Amount", simple_group='Zap Out Token')
             , title= "Tokens Withdrawn from yvBoost Vault via Zap"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='Amount (USD)')
fig.update_xaxes(title_text=None)
fig.show()

# Conclusions

We have taken a journey into the inner workings of the yvBoost vault, which has showed us how CRV tokens get locked in the Curve DAO contract, and how the yveCRV-DAO vault maximises the return from this strategy.  The yvBoost yVault automates this to make it a truly set & forget yield solution for the user.  We then saw how users deposited & withdrew from the yvBoost vault, and drilled into what tokens they used when they weren't working in yveCRV tokens.  The Zap transactions were mainly stablecoins on the way in, and ETH/WETH on the way out.

All data was sourced from the curated on-chain data tables at [Flipside Crypto](https://flipsidecrypto.com/)